In [ ]:
import pandas as pd
import random
from tabulate import tabulate
from datetime import datetime

# Load the Excel file
file_path = 'path your dataset'
classrooms_df = pd.read_excel(file_path, sheet_name='Classrooms')
fall_semester_df = pd.read_excel(file_path, sheet_name='Fall Semester')
spring_semester_df = pd.read_excel(file_path, sheet_name='Spring Semester')
students_df = pd.read_excel(file_path, sheet_name='Students')

# Remove extra spaces in column names
fall_semester_df.columns = fall_semester_df.columns.str.strip()
spring_semester_df.columns = spring_semester_df.columns.str.strip()

# Process the data
classrooms = classrooms_df[['Classroom', 'Capacity']].to_dict(orient='records')
courses_fall = fall_semester_df[['Course', 'Semester', 'ECTS']].to_dict(orient='records')
courses_spring = spring_semester_df[['Course', 'Semester', 'ECTS']].to_dict(orient='records')
students = students_df[['Student', 'Class']].to_dict(orient='records')

# Define elective courses
elective_courses = [
....
]

professional_courses = [
.....
]

general_culture_courses = [
.....
]

# Update the courses list by removing the placeholder elective course
courses_fall = [course for course in courses_fall if "Elective Courses for Field Education" not in course['Course'] and "Elective Vocational Knowledge" not in course['Course'] and "Elective Courses for General Culture" not in course['Course']]
courses_fall.extend(elective_courses)
courses_fall.extend(professional_courses)
courses_fall.extend(general_culture_courses)

# Update the courses list by removing the placeholder elective course
courses_spring = [course for course in courses_spring if "Elective Courses for Field Education" not in course['Course'] and "Elective Vocational Knowledge" not in course['Course'] and "Elective Courses for General Culture" not in course['Course']]
courses_spring.extend(elective_courses)
courses_spring.extend(professional_courses)
courses_spring.extend(general_culture_courses)

# Define time slots excluding the lunch break
all_time_slots = [
    ('08:30', '10:30'),
    ('11:00', '13:00'),
    ('13:30', '15:30'),
    ('16:00', '18:00')
]

# Exclude lunch break by filtering time intervals
time_slots = [(start, end) for start, end in all_time_slots if not ('11:00' <= start < '13:30')]
# Define weekdays
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Function to convert time strings to minutes
def time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

# Function to check if a time slot is available
def is_slot_available(occupied_slots, classroom, day, start_time, end_time, term_slots):
    start_minutes = time_to_minutes(start_time)
    end_minutes = time_to_minutes(end_time)
    
    # Check classroom availability
    for (start, end) in occupied_slots[day][classroom]:
        if not (end_minutes + 30 <= time_to_minutes(start) or start_minutes >= time_to_minutes(end) + 30):
            return False
    
    # Check term (semester) availability if term_slots is provided
    if term_slots:
        for (start, end) in term_slots[day]:
            if not (end_minutes + 30 <= time_to_minutes(start) or start_minutes >= time_to_minutes(end) + 30):
                return False
    
    return True



import numpy as np

# Initialize pheromone matrix
def initialize_pheromone_matrix(courses, time_slots):
    pheromone_matrix = {}
    for course in courses:
        for day in weekdays:
            for start_time, end_time in time_slots:
                pheromone_matrix[(course['Course'], day, start_time)] = 1.0
    return pheromone_matrix

# Function to construct a schedule by an ant
def construct_solution(pheromone_matrix, classrooms, courses, weekdays, time_slots):
    schedule = []
    occupied_slots = {day: {classroom['Classroom']: [] for classroom in classrooms} for day in weekdays}
    occupied_slots_by_term = {term: {day: [] for day in weekdays} for term in set(course['Semester'] for course in courses if 'Semester' in course)}
    
    for course in courses:
        assigned = False
        while not assigned:
            classroom = random.choice(classrooms)
            day = random.choice(weekdays)
            
            # Calculate probabilities for all time slots based on pheromone levels
            pheromone_levels = [(start_time, end_time, pheromone_matrix[(course['Course'], day, start_time)]) for start_time, end_time in time_slots]
            total_pheromone = sum(pheromone for _, _, pheromone in pheromone_levels)
            probabilities = [(start_time, end_time, pheromone / total_pheromone) for start_time, end_time, pheromone in pheromone_levels]
            
            # Select a time slot probabilistically
            rand = random.random()
            cumulative_probability = 0.0
            for start_time, end_time, probability in probabilities:
                cumulative_probability += probability
                if rand < cumulative_probability:
                    selected_time_slot = (start_time, end_time)
                    break
            
            start_time, end_time = selected_time_slot
            
            # Check if it's an elective course
            if 'Semester' not in course:
                term = None
                capacity_check = False  # No capacity check for elective courses
            else:
                term = course['Semester']
                capacity_check = True  # Capacity check for non-elective courses
            
            # Check if capacity check is required
            if capacity_check:
                # Calculate student count for the course
                if 'ECTS' in course:
                    student_count = sum(1 for student in students if student['Course'] == course['Semester'])
                else:
                    student_count = 0
                
                # Check if classroom capacity is sufficient
                if student_count > classroom['Capacity']:
                    continue  # Try again with a different slot if capacity is exceeded
            
            # Check if the time slot is available
            if is_slot_available(occupied_slots, classroom['Classroom'], day, start_time, end_time, occupied_slots_by_term.get(term)):
                occupied_slots[day][classroom['Classroom']].append((start_time, end_time))
                if term:
                    occupied_slots_by_term[term][day].append((start_time, end_time))
                schedule.append({
                    'Course': course['Course'],
                    'Class': course.get('Semester', 'Elective'),
                    'Classroom': classroom['Classroom'],
                    'Day': day,
                    'Start Time': start_time,
                    'End Time': end_time,
                    'Capacity': classroom['Capacity']
                })
                assigned = True
    
    return schedule

# Function to update pheromone levels
def update_pheromone_levels(pheromone_matrix, solutions, evaporation_rate=0.5, pheromone_deposit=1.0):
    # Evaporate pheromone levels
    for key in pheromone_matrix:
        pheromone_matrix[key] *= (1 - evaporation_rate)
    
    # Deposit pheromone based on solution quality
    for solution in solutions:
        conflicts = calculate_conflicts(solution)
        travel_time = calculate_travel_time(solution)
        quality = 1.0 / (conflicts + travel_time + 1.0)
        
        for course in solution:
            pheromone_matrix[(course['Course'], course['Day'], course['Start Time'])] += pheromone_deposit * quality
            
            
def calculate_conflicts(schedule):
    # Calculate total conflicts in the schedule
    conflicts = 0
    for i in range(len(schedule)):
        for j in range(i + 1, len(schedule)):
            if courses_overlap(schedule[i], schedule[j]):
                conflicts += 1
    return conflicts

def calculate_travel_time(schedule):
    # Calculate total travel time for students
    travel_time = 0
    for student in students:
        student_courses = [course for course in schedule if course['Class'] == student['Class']]
        if len(student_courses) > 1:
            travel_time += calculate_student_travel_time(student_courses)
    return travel_time

def calculate_student_travel_time(student_courses):
    # Calculate travel time for a student between consecutive courses
    total_time = 0
    for i in range(len(student_courses) - 1):
        end_time_current = datetime.strptime(student_courses[i]['End Time'], '%H:%M')
        start_time_next = datetime.strptime(student_courses[i+1]['Start Time'], '%H:%M')
        if start_time_next > end_time_current:
            total_time += (start_time_next - end_time_current).seconds / 60
    return total_time

def courses_overlap(course1, course2):
    # Check if two courses overlap in time and classroom
    if (course1['Day'] == course2['Day']):
        #and
        #course1['Derslik'] == course2['Derslik']):
        start_time1 = datetime.strptime(course1['Start Time'], '%H:%M')
        end_time1 = datetime.strptime(course1['End Time'], '%H:%M')
        start_time2 = datetime.strptime(course2['Start Time'], '%H:%M')
        end_time2 = datetime.strptime(course2['End Time'], '%H:%M')

        if (start_time1 < end_time2 and start_time2 < end_time1):
            return True
    return False

# Ant Colony Optimization function
def ant_colony_optimization(iterations, num_ants, classrooms, courses, weekdays, time_slots):
    pheromone_matrix = initialize_pheromone_matrix(courses, time_slots)
    
    best_solution = None
    best_fitness = float('inf')
    
    for iteration in range(iterations):
        solutions = []
        for ant in range(num_ants):
            solution = construct_solution(pheromone_matrix, classrooms, courses, weekdays, time_slots)
            solutions.append(solution)
        
        # Evaluate solutions
        for solution in solutions:
            conflicts = calculate_conflicts(solution)
            travel_time = calculate_travel_time(solution)
            fitness =  conflicts + travel_time
            
            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = solution
        
        # Update pheromone levels
        update_pheromone_levels(pheromone_matrix, solutions)
        
        print(f"Iteration {iteration + 1}/{iterations}, Best Fitness: {best_fitness}")
    
    return best_solution, best_fitness

# Parameters
iterations = 10  # Number of iterations
num_ants = 50  # Number of ants

# Run ACO for Güz
print("Fall semester iteration results:")
best_schedule_fall, best_fitness_fall = ant_colony_optimization(iterations, num_ants, classrooms, courses_fall, weekdays, time_slots)
print(f"Best Fitness for Fall Semester: {best_fitness_fall}")
print(tabulate(best_schedule_fall, headers="keys", tablefmt="grid"))

# Run ACO for Bahar
print("Spring term iteration results:")
best_schedule_spring, best_fitness_spring = ant_colony_optimization(iterations, num_ants, classrooms, courses_spring, weekdays, time_slots)
print(f"Best Fitness for Spring Semester: {best_fitness_spring}")
print(tabulate(best_schedule_spring, headers="keys", tablefmt="grid"))

# A LOW FITNESS VALUE SHOWS THAT IT IS A BETTER COURSE PROGRAM.